## Updated Lens Generation 02/10/24
- Here, we make lenses that have consistent sheared camera view
- We also add edges to the lenses (no we actually don't)
- Finally, we render views of Eric's lenses for End-to-End and DINO

In [ ]:
import numpy as np 
import matplotlib.pyplot as plt
import plotly
from leyla_fns import *
from cleanplots import *

In [ ]:
import numpy as np
import plotly.graph_objects as go
from scipy.ndimage import gaussian_filter

### Make PSF flat images

In [ ]:
diffuser_psf = skimage.io.imread('/home/lkabuli_waller/workspace/EncodingInformation/imager_experiments/psfs/diffuser_psf.png')
diffuser_psf = diffuser_psf[:,:,1]
diffuser_resize = diffuser_psf[200:500, 250:550]
diffuser_resize = resize(diffuser_resize, (400, 400), anti_aliasing=True)  #resize(diffuser_psf, (28, 28))
diffuser_region = diffuser_resize[:32*4, :32*4]
diffuser_region /=  np.sum(diffuser_region)

In [ ]:
import matplotlib.colors as mcolors 

lime = mcolors.LinearSegmentedColormap.from_list("custom_cmap", ['black', 'lime'])

In [ ]:
# green psfs 
diffuser_psf = load_diffuser_32()
four_psf = load_four_lens_32()
one_psf = load_single_lens_32()
plt.figure(figsize=(10, 4))
plt.subplot(1,3,1)
plt.imshow(one_psf, cmap=lime, interpolation='spline36')
plt.title('Single Lens')
# clear the axes
plt.gca().set_xticks([])
plt.gca().set_yticks([])
plt.subplot(1,3,2)
plt.imshow(four_psf, cmap=lime, interpolation='spline36')
plt.title('Four Lenses')
# clear the axes
plt.gca().set_xticks([])
plt.gca().set_yticks([])
plt.subplot(1,3,3)
plt.imshow(diffuser_region, cmap=lime, interpolation='spline36')
plt.title('Diffuser')
# clear the axes
plt.gca().set_xticks([])
plt.gca().set_yticks([])

#plt.savefig('01_11_2024_figures/green_planar_psfs.pdf', transparent=True, bbox_inches='tight')

In [ ]:
# load the PSFs

diffuser_psf = load_diffuser_32()
four_psf = load_four_lens_32()
one_psf = load_single_lens_32()
plt.figure(figsize=(10, 4))
plt.subplot(1,3,1)
plt.imshow(one_psf, cmap='inferno', interpolation='spline36')
plt.title('Single Lens')
# clear the axes
plt.gca().set_xticks([])
plt.gca().set_yticks([])
plt.subplot(1,3,2)
plt.imshow(four_psf, cmap='inferno', interpolation='spline36')
plt.title('Four Lenses')
# clear the axes
plt.gca().set_xticks([])
plt.gca().set_yticks([])
plt.subplot(1,3,3)
plt.imshow(diffuser_region, cmap='inferno', interpolation='spline36')
plt.title('Diffuser')
# clear the axes
plt.gca().set_xticks([])
plt.gca().set_yticks([])

#plt.savefig('01_11_2024_figures/planar_psfs.pdf', transparent=True, bbox_inches='tight')

In [ ]:
z_four_lens = np.zeros((500, 500))
z_four_lens[375, 125] = 1 
z_four_lens[220, 175] = 1
z_four_lens[100, 250] = 1
z_four_lens[350, 380] = 1
z_four_lens = z_four_lens * 1
z_four_lens = gaussian_filter(z_four_lens, sigma=40)
z_four_lens = np.rot90(z_four_lens, k=-1)
plt.imshow(z_four_lens)
plt.colorbar()

## Four Lens

In [ ]:

# Define the surface geometry
x = np.linspace(-1, 1, 500)
y = np.linspace(-1, 1, 500)
x_grid, y_grid = np.meshgrid(y, x)
r = np.sqrt(x_grid ** 2 + y_grid ** 2)
z_diffuser = z_four_lens * -1
# make circular instead of square
z_diffuser[r > 1] = np.nan 

lighting = dict(ambient=0, fresnel=3, roughness=0.75, specular=0.5) # specular will make it brighter/saturat
lightposition=dict(x=5, y=5, z=-2)
color_name = 'lightblue' 
opacity = 0.95

# Create a 3D plot
fig = go.Figure()
fig.add_surface(
    x=x, 
    y=y, 
    z=z_diffuser, 
    colorscale=[[0, color_name], [1, color_name]],
    showscale=False, 
    opacity=opacity, lighting=lighting, lightposition=lightposition
)

# Update layout for camera, lighting, and axes
fig.update_layout(
    scene=dict(
        xaxis=dict(showline=False, zeroline=False, showticklabels=False, showgrid=False, backgroundcolor='rgba(0,0,0,0)'),
        yaxis=dict(showline=False, zeroline=False, showticklabels=False, showgrid=False, backgroundcolor='rgba(0,0,0,0)'),
        zaxis=dict(showline=False, zeroline=False, showticklabels=False, showgrid=False, backgroundcolor='rgba(0,0,0,0)'),
        aspectmode='manual',
        aspectratio=dict(x=1, y=1, z=0.1)
    ),
    paper_bgcolor='white',
    plot_bgcolor='white'
)
  
# turn off axis labels
fig.update_layout(scene=dict(xaxis=dict(title=''), yaxis=dict(title=''),  zaxis=dict(title='')))
# Update the camera to rotate the plot
fig.update_layout(
    scene_camera=dict(
        eye=dict(x=0.7*0.7, y=-1.9*0.7, z=-1.4*0.7), # rotated to other direction?
        up=dict(x=1, y=0, z=0)            # 'up' direction
    ),
)
fig.show()
fig.write_image('01_11_2024_figures/02_12_four_lens_render.pdf', scale=10)

## Single Lens

In [ ]:
import numpy as np
import plotly.graph_objects as go
from scipy.ndimage import gaussian_filter
# Define the surface geometry
x = np.linspace(-1, 1, 500)
y = np.linspace(-1, 1, 500)
x_grid, y_grid = np.meshgrid(y, x)
r = np.sqrt(x_grid ** 2 + y_grid ** 2)
z_diffuser = gaussian_filter(r, sigma=15)
# make circular instead of square
z_diffuser[r > 1] = np.nan

lighting = dict(ambient=0, fresnel=3, roughness=0.75, specular=0.5) # specular will make it brighter/saturat
lightposition=dict(x=5, y=5, z=-2)
color_name = 'lightblue' #'white' #'lightblue'
opacity=0.95

# Create a 3D plot
fig = go.Figure()
fig.add_surface(
    x=x, 
    y=y, 
    z=z_diffuser, 
    colorscale=[[0, color_name], [1, color_name]], 
    showscale=False, 
    opacity=opacity, lighting=lighting, lightposition=lightposition
)

# Update layout for camera, lighting, and axes
fig.update_layout(
    scene=dict(
        xaxis=dict(showline=False, zeroline=False, showticklabels=False, showgrid=False, backgroundcolor='rgba(0,0,0,0)'),
        yaxis=dict(showline=False, zeroline=False, showticklabels=False, showgrid=False, backgroundcolor='rgba(0,0,0,0)'),
        zaxis=dict(showline=False, zeroline=False, showticklabels=False, showgrid=False, backgroundcolor='rgba(0,0,0,0)'),
        aspectmode='manual',
        aspectratio=dict(x=1, y=1, z=0.1)
    ),
    paper_bgcolor='white',
    plot_bgcolor='white'
)
  
# turn off axis labels
fig.update_layout(scene=dict(xaxis=dict(title=''), yaxis=dict(title=''),  zaxis=dict(title='')))
# Update the camera to rotate the plot
fig.update_layout(
    scene_camera=dict(
        eye=dict(x=0.7*0.7, y=-1.9*0.7, z=-1.4*0.7), # rotated to other direction?
        up=dict(x=1, y=0, z=0)            # 'up' direction
    ),
)
fig.show()
fig.write_image('01_11_2024_figures/02_12_one_lens_render.pdf', scale=10)

## Make lens renders (dino) for eric

In [ ]:
folder = 'e2e_full_volume_phase_masks/high_res_dino/'
initial = np.load(folder + 'checkpoint_0.npy')
middle = np.load(folder +'checkpoint_120.npy')
final = np.load(folder + 'checkpoint_1080.npy')


In [ ]:
import numpy as np
import plotly.graph_objects as go
from scipy.ndimage import gaussian_filter
# Define the surface geometry
x = np.linspace(-1, 1, 500)
y = np.linspace(-1, 1, 500)
x_grid, y_grid = np.meshgrid(y, x)
r = np.sqrt(x_grid ** 2 + y_grid ** 2)
z_diffuser = -15 * np.copy(initial) 
# make circular instead of square
z_diffuser[r > 1] = np.nan

lighting = dict(ambient=0, fresnel=3, roughness=0.75, specular=0.5) # specular will make it brighter/saturat
lightposition=dict(x=5, y=5, z=-2)
color_name = 'lightblue'
opacity=0.95

# Create a 3D plot
fig = go.Figure()
fig.add_surface(
    x=x, 
    y=y, 
    z=z_diffuser, 
    colorscale=[[0, color_name], [1, color_name]], 
    showscale=False, 
    opacity=opacity, lighting=lighting, lightposition=lightposition
)

# Update layout for camera, lighting, and axes
fig.update_layout(
    scene=dict(
        xaxis=dict(showline=False, zeroline=False, showticklabels=False, showgrid=False, backgroundcolor='rgba(0,0,0,0)'),
        yaxis=dict(showline=False, zeroline=False, showticklabels=False, showgrid=False, backgroundcolor='rgba(0,0,0,0)'),
        zaxis=dict(showline=False, zeroline=False, showticklabels=False, showgrid=False, backgroundcolor='rgba(0,0,0,0)'),
        aspectmode='manual',
        aspectratio=dict(x=1, y=1, z=0.1)
    ),
    paper_bgcolor='white',
    plot_bgcolor='white'
)
  
# turn off axis labels
fig.update_layout(scene=dict(xaxis=dict(title=''), yaxis=dict(title=''),  zaxis=dict(title='')))
# Update the camera to rotate the plot
fig.update_layout(
    scene_camera=dict(
        eye=dict(x=0.7*0.7, y=-1.9*0.7, z=-1.4*0.7), # rotated to other direction?
        up=dict(x=1, y=0, z=0)            # 'up' direction
    ),
)
fig.show()
fig.write_image('01_11_2024_figures/02_12_dino_initial_lens_render.pdf', scale=10)

In [ ]:
import numpy as np
import plotly.graph_objects as go
from scipy.ndimage import gaussian_filter
# Define the surface geometry
x = np.linspace(-1, 1, 500)
y = np.linspace(-1, 1, 500)
x_grid, y_grid = np.meshgrid(y, x)
r = np.sqrt(x_grid ** 2 + y_grid ** 2)
z_diffuser = -15 * np.copy(middle)
# make circular instead of square
z_diffuser[r > 1] = np.nan

lighting = dict(ambient=0, fresnel=3, roughness=0.75, specular=0.5) # specular will make it brighter/saturat
lightposition=dict(x=5, y=5, z=-2)
color_name = 'lightblue' #'white' #'lightblue'
opacity=0.95

# Create a 3D plot
fig = go.Figure()
fig.add_surface(
    x=x, 
    y=y, 
    z=z_diffuser, 
    colorscale=[[0, color_name], [1, color_name]], 
    showscale=False, 
    opacity=opacity, lighting=lighting, lightposition=lightposition
)

# Update layout for camera, lighting, and axes
fig.update_layout(
    scene=dict(
        xaxis=dict(showline=False, zeroline=False, showticklabels=False, showgrid=False, backgroundcolor='rgba(0,0,0,0)'),
        yaxis=dict(showline=False, zeroline=False, showticklabels=False, showgrid=False, backgroundcolor='rgba(0,0,0,0)'),
        zaxis=dict(showline=False, zeroline=False, showticklabels=False, showgrid=False, backgroundcolor='rgba(0,0,0,0)'),
        aspectmode='manual',
        aspectratio=dict(x=1, y=1, z=0.1)
    ),
    paper_bgcolor='white',
    plot_bgcolor='white'
)
  
# turn off axis labels
fig.update_layout(scene=dict(xaxis=dict(title=''), yaxis=dict(title=''),  zaxis=dict(title='')))
# Update the camera to rotate the plot
fig.update_layout(
    scene_camera=dict(
        eye=dict(x=0.7*0.7, y=-1.9*0.7, z=-1.4*0.7), # rotated to other direction?
        up=dict(x=1, y=0, z=0)            # 'up' direction
    ),
)
fig.show()
fig.write_image('01_11_2024_figures/02_12_dino_middle_lens_render.pdf', scale=10)

In [ ]:
import numpy as np
import plotly.graph_objects as go
from scipy.ndimage import gaussian_filter
# Define the surface geometry
x = np.linspace(-1, 1, 500)
y = np.linspace(-1, 1, 500)
x_grid, y_grid = np.meshgrid(y, x)
r = np.sqrt(x_grid ** 2 + y_grid ** 2)
z_diffuser = -15 * np.copy(final)
# make circular instead of square
z_diffuser[r > 1] = np.nan

lighting = dict(ambient=0, fresnel=3, roughness=0.75, specular=0.5) # specular will make it brighter/saturat
lightposition=dict(x=5, y=5, z=-2)
color_name = 'lightblue' #'white' #'lightblue'
opacity=0.95

# Create a 3D plot
fig = go.Figure()
fig.add_surface(
    x=x, 
    y=y, 
    z=z_diffuser, 
    colorscale=[[0, color_name], [1, color_name]], 
    showscale=False, 
    opacity=opacity, lighting=lighting, lightposition=lightposition
)

# Update layout for camera, lighting, and axes
fig.update_layout(
    scene=dict(
        xaxis=dict(showline=False, zeroline=False, showticklabels=False, showgrid=False, backgroundcolor='rgba(0,0,0,0)'),
        yaxis=dict(showline=False, zeroline=False, showticklabels=False, showgrid=False, backgroundcolor='rgba(0,0,0,0)'),
        zaxis=dict(showline=False, zeroline=False, showticklabels=False, showgrid=False, backgroundcolor='rgba(0,0,0,0)'),
        aspectmode='manual',
        aspectratio=dict(x=1, y=1, z=0.1)
    ),
    paper_bgcolor='white',
    plot_bgcolor='white'
)
  
# turn off axis labels
fig.update_layout(scene=dict(xaxis=dict(title=''), yaxis=dict(title=''),  zaxis=dict(title='')))
# Update the camera to rotate the plot
fig.update_layout(
    scene_camera=dict(
        eye=dict(x=0.7*0.7, y=-1.9*0.7, z=-1.4*0.7), # rotated to other direction?
        up=dict(x=1, y=0, z=0)            # 'up' direction
    ),
)
fig.show()
fig.write_image('01_11_2024_figures/02_12_dino_final_lens_render.pdf', scale=10)

## Make lens renders for eric

In [ ]:
folder = 'e2e_full_volume_phase_masks/phase_masks_high_res/'
initial = np.load(folder + 'checkpoint_0.npy')
middle = np.load(folder +'checkpoint_300.npy')
final = np.load(folder + 'checkpoint_5660.npy')


In [ ]:
import numpy as np
import plotly.graph_objects as go
from scipy.ndimage import gaussian_filter
# Define the surface geometry
x = np.linspace(-1, 1, 500)
y = np.linspace(-1, 1, 500)
x_grid, y_grid = np.meshgrid(y, x)
r = np.sqrt(x_grid ** 2 + y_grid ** 2)
z_diffuser = -15 * np.copy(initial) 
# make circular instead of square
z_diffuser[r > 1] = np.nan

lighting = dict(ambient=0, fresnel=3, roughness=0.75, specular=0.5) # specular will make it brighter/saturat
lightposition=dict(x=5, y=5, z=-2)
color_name = 'lightblue'
opacity=0.95

# Create a 3D plot
fig = go.Figure()
fig.add_surface(
    x=x, 
    y=y, 
    z=z_diffuser, 
    colorscale=[[0, color_name], [1, color_name]], 
    showscale=False, 
    opacity=opacity, lighting=lighting, lightposition=lightposition
)

# Update layout for camera, lighting, and axes
fig.update_layout(
    scene=dict(
        xaxis=dict(showline=False, zeroline=False, showticklabels=False, showgrid=False, backgroundcolor='rgba(0,0,0,0)'),
        yaxis=dict(showline=False, zeroline=False, showticklabels=False, showgrid=False, backgroundcolor='rgba(0,0,0,0)'),
        zaxis=dict(showline=False, zeroline=False, showticklabels=False, showgrid=False, backgroundcolor='rgba(0,0,0,0)'),
        aspectmode='manual',
        aspectratio=dict(x=1, y=1, z=0.1)
    ),
    paper_bgcolor='white',
    plot_bgcolor='white'
)
  
# turn off axis labels
fig.update_layout(scene=dict(xaxis=dict(title=''), yaxis=dict(title=''),  zaxis=dict(title='')))
# Update the camera to rotate the plot
fig.update_layout(
    scene_camera=dict(
        eye=dict(x=0.7*0.7, y=-1.9*0.7, z=-1.4*0.7), # rotated to other direction?
        up=dict(x=1, y=0, z=0)            # 'up' direction
    ),
)
fig.show()
fig.write_image('01_11_2024_figures/02_12_initial_lens_render.pdf', scale=10)

In [ ]:
import numpy as np
import plotly.graph_objects as go
from scipy.ndimage import gaussian_filter
# Define the surface geometry
x = np.linspace(-1, 1, 500)
y = np.linspace(-1, 1, 500)
x_grid, y_grid = np.meshgrid(y, x)
r = np.sqrt(x_grid ** 2 + y_grid ** 2)
z_diffuser = -15 * np.copy(middle)
# make circular instead of square
z_diffuser[r > 1] = np.nan

lighting = dict(ambient=0, fresnel=3, roughness=0.75, specular=0.5) # specular will make it brighter/saturat
lightposition=dict(x=5, y=5, z=-2)
color_name = 'lightblue' #'white' #'lightblue'
opacity=0.95

# Create a 3D plot
fig = go.Figure()
fig.add_surface(
    x=x, 
    y=y, 
    z=z_diffuser, 
    colorscale=[[0, color_name], [1, color_name]], 
    showscale=False, 
    opacity=opacity, lighting=lighting, lightposition=lightposition
)

# Update layout for camera, lighting, and axes
fig.update_layout(
    scene=dict(
        xaxis=dict(showline=False, zeroline=False, showticklabels=False, showgrid=False, backgroundcolor='rgba(0,0,0,0)'),
        yaxis=dict(showline=False, zeroline=False, showticklabels=False, showgrid=False, backgroundcolor='rgba(0,0,0,0)'),
        zaxis=dict(showline=False, zeroline=False, showticklabels=False, showgrid=False, backgroundcolor='rgba(0,0,0,0)'),
        aspectmode='manual',
        aspectratio=dict(x=1, y=1, z=0.1)
    ),
    paper_bgcolor='white',
    plot_bgcolor='white'
)
  
# turn off axis labels
fig.update_layout(scene=dict(xaxis=dict(title=''), yaxis=dict(title=''),  zaxis=dict(title='')))
# Update the camera to rotate the plot
fig.update_layout(
    scene_camera=dict(
        eye=dict(x=0.7*0.7, y=-1.9*0.7, z=-1.4*0.7), # rotated to other direction?
        up=dict(x=1, y=0, z=0)            # 'up' direction
    ),
)
fig.show()
fig.write_image('01_11_2024_figures/02_12_middle_lens_render.pdf', scale=10)

In [ ]:
import numpy as np
import plotly.graph_objects as go
from scipy.ndimage import gaussian_filter
# Define the surface geometry
x = np.linspace(-1, 1, 500)
y = np.linspace(-1, 1, 500)
x_grid, y_grid = np.meshgrid(y, x)
r = np.sqrt(x_grid ** 2 + y_grid ** 2)
z_diffuser = -15 * np.copy(final)
# make circular instead of square
z_diffuser[r > 1] = np.nan

lighting = dict(ambient=0, fresnel=3, roughness=0.75, specular=0.5) # specular will make it brighter/saturat
lightposition=dict(x=5, y=5, z=-2)
color_name = 'lightblue' #'white' #'lightblue'
opacity=0.95

# Create a 3D plot
fig = go.Figure()
fig.add_surface(
    x=x, 
    y=y, 
    z=z_diffuser, 
    colorscale=[[0, color_name], [1, color_name]], 
    showscale=False, 
    opacity=opacity, lighting=lighting, lightposition=lightposition
)

# Update layout for camera, lighting, and axes
fig.update_layout(
    scene=dict(
        xaxis=dict(showline=False, zeroline=False, showticklabels=False, showgrid=False, backgroundcolor='rgba(0,0,0,0)'),
        yaxis=dict(showline=False, zeroline=False, showticklabels=False, showgrid=False, backgroundcolor='rgba(0,0,0,0)'),
        zaxis=dict(showline=False, zeroline=False, showticklabels=False, showgrid=False, backgroundcolor='rgba(0,0,0,0)'),
        aspectmode='manual',
        aspectratio=dict(x=1, y=1, z=0.1)
    ),
    paper_bgcolor='white',
    plot_bgcolor='white'
)
  
# turn off axis labels
fig.update_layout(scene=dict(xaxis=dict(title=''), yaxis=dict(title=''),  zaxis=dict(title='')))
# Update the camera to rotate the plot
fig.update_layout(
    scene_camera=dict(
        eye=dict(x=0.7*0.7, y=-1.9*0.7, z=-1.4*0.7), # rotated to other direction?
        up=dict(x=1, y=0, z=0)            # 'up' direction
    ),
)
fig.show()
fig.write_image('01_11_2024_figures/02_12_final_lens_render.pdf', scale=10)

## Diffuser without black aperture and smaller scale features (zoomed in)
Eventually want to make it a circle, so want to have the edges on the right side facing out and the top side facing out (towards the right)

In [ ]:
import numpy as np
import plotly.graph_objects as go
from scipy.ndimage import gaussian_filter
# Define the surface geometry
x = np.linspace(-1, 1, 500)
y = np.linspace(-1, 1, 500)
x_grid, y_grid = np.meshgrid(y, x)
r = np.sqrt(x_grid ** 2 + y_grid ** 2)
z_diffuser = gaussian_filter(np.random.rand(y.size, x.size), sigma=25) # make it taller by multiplying by value less than 1
# make circular instead of square
z_diffuser[r > 1] = np.nan 

lighting = dict(ambient=0, fresnel=3, roughness=0.75, specular=0.5) # specular will make it brighter/saturat
lightposition=dict(x=5, y=5, z=-2)
color_name = 'lightblue' #'white' #'lightblue'
opacity=0.95

# Create a 3D plot
fig = go.Figure()
fig.add_surface(
    x=x, 
    y=y, 
    z=z_diffuser, 
    colorscale=[[0, color_name], [1, color_name]], 
    showscale=False, 
    opacity=opacity, lighting=lighting, lightposition=lightposition
)

# Update layout for camera, lighting, and axes
fig.update_layout(
    scene=dict(
        xaxis=dict(showline=False, zeroline=False, showticklabels=False, showgrid=False, backgroundcolor='rgba(0,0,0,0)'),
        yaxis=dict(showline=False, zeroline=False, showticklabels=False, showgrid=False, backgroundcolor='rgba(0,0,0,0)'),
        zaxis=dict(showline=False, zeroline=False, showticklabels=False, showgrid=False, backgroundcolor='rgba(0,0,0,0)'),
        aspectmode='manual',
        aspectratio=dict(x=1, y=1, z=0.1)
    ),
    paper_bgcolor='white',
    plot_bgcolor='white'
)
  
# turn off axis labels
fig.update_layout(scene=dict(xaxis=dict(title=''), yaxis=dict(title=''),  zaxis=dict(title='')))
# Update the camera to rotate the plot
fig.update_layout(
    scene_camera=dict(
        eye=dict(x=0.7*0.7, y=-1.9*0.7, z=1.4*0.7), # rotated to other direction?
        up=dict(x=1, y=0, z=0)            # 'up' direction
    ),
)
fig.show()
fig.write_image('01_11_2024_figures/02_12_diffuser_render.pdf', scale=10)